# micrograd

> Minimal implementation of autograd

In [ ]:
#|default_exp micrograd

In [ ]:
#| export
import numpy as np
import matplotlib.pyplot as plt
from fastcore.utils import L

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
print("Hello")

Hello


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()